In [2]:
import spotipy
import os
import sys 
import webbrowser
import spotipy.util as util
from json.decoder import JSONDecodeError
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn import preprocessing
import math

# Create recommended tracks

Get authorization token to access the Spotify API. To get a the credentials to make authorized calls (client ID and client secret), you have to register your app at 'My Dashboard'. 

In [3]:
# Get recently played tracks
username = ''
scope = 'user-read-recently-played'
client_id = ''
client_secret = ''
redirect_uri='https://google.com/'
token = util.prompt_for_user_token(username,
                           scope,
                           client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri='https://google.com/')

if token:
    sp = spotipy.Spotify(auth=token)

# Create Song Recommendation Function

With the funtion recommend_songs() the user specifies artists and / or songs they like to create a list of recommended songs. 
- Input for the artist argument should be in list format. Ex: ['Justin Bieber, 'Drake', 'Beyonce'...]
- Input for the songs arguments should be a dictionary of tracks and the track artists. Ex: {'thank u, next':'Ariana Grande', 'Watermelon Sugar':'Harry Styles'...}
- The desired number of recomended songs in increments of 20 can be specified with the num_songs argument. The default is 20 songs. 
- The number of artist inputs + the number of song inputs can not exceed 5.

In [16]:
def recommend_songs(artists = [], songs = [], num_songs = 20):

    if (artists is not None) & (type(artists) != list):
        return "Artist names must be a list. Ex: ['Justin Bieber', 'Beyonce', 'Ed Sheeran'...]"
    
    if (len(songs) >=1) & (type(songs) != dict):
        return "Songs must be in a dictionary format. Ex: {'thank u, next': 'Ariana Grande', 'Nice For What': 'Drake'...}"
    
    if (len(artists) + len(songs)) > 5:
        return "Too many artists or songs provided. The number of songs plus artists can't exceed 5."
    
    # Create dataframe for playlist tracks
    df = pd.DataFrame(columns=['track', 'artists', 'id'])
    desired_runs = (num_songs / 20) # Every iteration 20 songs are added
    num_of_runs = 0 
    song_dict = songs

    while num_of_runs < desired_runs: 
        
        # If both artists and songs are specified
        if (len(artists) >= 1) & (len(songs) >= 1):
            seed_artists = [sp.search(q='artist:' + i, type='artist')['artists']['items'][0]['id'] for i in artists]
            seed_tracks = [sp.search(q='artist:' + song_dict[i] + ' track:' + i, type='track')['tracks']['items'][0]['id'] for i in song_dict]
            recommended_tracks = sp.recommendations(seed_artists=seed_artists, seed_tracks = seed_tracks)
            if num_of_runs == 0: # Only print string on the first iteration
                print(f'Generating a playlist of {num_songs} songs based on the the artists {artists} and the songs {[song for song in song_dict]}.')
            
        # If only artist names are specified
        elif (len(artists) >= 1):
            seed_artists = [sp.search(q='artist:' + i, type='artist')['artists']['items'][0]['id'] for i in artists]
            recommended_tracks = sp.recommendations(seed_artists=seed_artists)
            if num_of_runs == 0:
                print(f'Generating a playlist of {num_songs} songs based on the artists {artists}.')
            
        #If only track names are specified
        else:
            seed_tracks = [sp.search(q='artist:' + song_dict[i] + ' track:' + i, type='track')['tracks']['items'][0]['id'] for i in song_dict]
            recommended_tracks = sp.recommendations(seed_tracks=seed_tracks)
            if num_of_runs == 0:
                print(f'Generating a playlist of {num_songs} songs that are similar to {[song for song in song_dict]}.')
            
        # Record track name, track id, and track artist for each recommended track
        tracks = [recommended_tracks['tracks'][track]['name'] for track in range(len(recommended_tracks['tracks']))]
        ids = [recommended_tracks['tracks'][track]['id'] for track in range(len(recommended_tracks['tracks']))]
        artists_list = []
        for track_index in range(len(recommended_tracks['tracks'])):
            artists_on_track = []
            for i in range(len(recommended_tracks['tracks'][track_index]['artists'])):
                artists_on_track.append(recommended_tracks['tracks'][track_index]['artists'][i]['name'])
            artists_list.append(artists_on_track)

        # Populate dataframe with recommended tracks   
        df = df.append(pd.DataFrame({'track':tracks, 'artists': artists_list, 'id' : ids}))
        num_of_runs += 1 

    return df

Let's try out the function. We will generate a playlist of 40 songs that are have similarities to the the music created by Shawn Mendes and Sam Smith. We also want our playlist to have songs similar to 'Stuck with U' by Ariana Grande and 'lovely' by Billie Eilish. 

In [18]:
df = recommend_songs(artists = ['Shawn Mendes', 'Sam Smith'],
                     songs = {'Stuck with U': 'Ariana Grande', 'lovely' : 'Billie Eilish'},
                     num_songs = 40)
df

Generating a playlist of 40 songs based on the the artists ['Shawn Mendes', 'Sam Smith'] and the songs ['Stuck with U', 'lovely'].


,track,artists,id
0,Roses,[Shawn Mendes],4uzUuDjoufcBXYZ27CsLIF
1,Stay With Me,[Sam Smith],5Nm9ERjJZ5oyfXZTECKmRt
2,betty,[Taylor Swift],5kI4eCXXzyuIUXjQra0Cxi
3,"Let Me Go (with Alesso, Florida Georgia Line &...","[Hailee Steinfeld, Alesso, Florida Georgia Lin...",5Gu0PDLN4YJeW75PpBSg9p
4,Intro,[Shawn Mendes],6VtNRkDKegG5dhGHVptgDP
5,How Do You Sleep?,[Sam Smith],34pRqzRiFcIuyRMuuEayzY
6,Good Thing (with Kehlani),"[Zedd, Kehlani]",5aUSEPNd3m5xliFK4pn5mU
7,Daisies,[Katy Perry],367wyLNqQMr5e8S2E6Zvpp
8,Supermarket Flowers,[Ed Sheeran],4VuS959DSpr82t3qBqCrWG
9,Get You The Moon (feat. Snøw),"[Kina, Snøw]",4ZLzoOkj0MPWrTLvooIuaa


If we are satisfied with the song recommendations, we can create a Spotify playlist!

# Create playlist function

The function create_playlist() will take a dataframe of songs and create a Spotify playlist using the Spotify API. This funtion requires a user ID which can be found on your Spotify profile page. Select the three dots that appear below your profile picture, select 'Share' and then 'Copy Spotify URI'. 

The arguments 'playlist_name' and 'df' must also be specified.

In [21]:
user = ''

def create_playlist(user, playlist_name, df):
    user = user
    playlist_df = df
    playlist_name = playlist_name

    # Create new playlist
    sp.user_playlist_create(user = user, name=playlist_name)


    # Create a dictionary of the user's playlist names and their associated playlistIDs
    user_playlists = sp.user_playlists(user = user) 
    for i in range(len(user_playlists['items'])):
        if user_playlists['items'][i]['name'] == playlist_name:
            playlistID = user_playlists['items'][i]['id']

    start = 0
    finish = 50
    while ((len(playlist_df) - finish)+50) > 50:
        tracks = list(playlist_df.id[start:finish])
        sp.user_playlist_add_tracks(user, playlistID, tracks, position = None)
        start += 50
        finish += 50
    else:
        tracks = list(playlist_df.id[start:])
        sp.user_playlist_add_tracks(user, playlistID, tracks, position = None)
        print(f'{len(playlist_df)} tracks were added to {playlist_name}.')

We can take the dataframe of recommended songs we generated and easily create a playlist that will appear on Spotify. I will call this playlist 'My Recommended Songs'.

In [22]:
create_playlist(user = '', playlist_name = 'My Recommended Songs', df = df)

40 tracks were added to My Recommended Songs.
